In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta, UTC
import pandas as pd
import numpy as np

In [2]:
import __init__
from app.utils.config import settings

# gather data

In [3]:
client = MongoClient(settings.mongo_uri)
client.test

Database(MongoClient(host=['newserver:27017'], document_class=dict, tz_aware=False, connect=True), 'test')

In [4]:
db = client["new_db"]
collection = db["divar-documents"]

In [5]:
start_date = datetime.now(UTC) - timedelta(days=14)
query = {
    "last_modified": {"$gte": start_date},
    "category": "apartment-sell",
    'district_id': '90',
}

In [6]:
results = collection.find(query)
df = pd.DataFrame(list(results))

In [7]:
df.head()

,_id,urls,description,district_id,district_name,location,title,features,unavailable_after,brand_model,...,district,gender,image_count,originality,price,rent,status,token,content_url,last_modified
0,67c41f0e5d12a3e439a6480e,[https://postimage01.divarcdn.com/static/photo...,✅️فرعی دنج ابن سینا ، غرق در نور\n\n❇️نقشه تفک...,90,فروش آپارتمان در یوسف‌آباد,{'exact_data': {'point': {'latitude': 35.73410...,یوسف اباد ابن سینا ((۱۷۰ متر)) ۳ خواب فول امکانات,"{'area': 170, 'production_year': 1393, 'rooms'...",2025-04-02T12:29:44.575497,,...,yousef-abad,,6,,32999999488,0,,wZCbs2Bt,https://api.divar.ir/v8/posts-v2/web/wZCbs2Bt,2025-03-02 09:04:12.795
1,67c41fad20b36e8a6f07ca0e,NaN,۶۰متر ۲خواب \nبدون امکانات \nطبقه دوم,90,فروش آپارتمان در یوسف‌آباد,NaN,فروش آپارتمان ۶۰ متری ۲ خوابه طبقه ۲ در یوسف‌آباد,"{'area': 60, 'production_year': 1379, 'rooms':...",2025-04-02T12:34:55.690186,,...,yousef-abad,,0,,8800000000,0,,wZE_xDRY,https://api.divar.ir/v8/posts-v2/web/wZE_xDRY,2025-03-02 09:06:52.686
2,67c43485b45f017c8c24d9b8,NaN,وقت به خیر :\n\n♦️با احترام ؛ به پیامک و چت پا...,90,فروش آپارتمان در یوسف‌آباد,{'exact_data': {'point': {'latitude': 35.73452...,۱ از ۳ قدرالسهم دار .۲۴۴ متر بنا,"{'area': 244, 'production_year': 1371, 'rooms'...",2025-03-14T14:00:04.422638,,...,yousef-abad,,0,,38000001024,0,,wZraCiSl,https://api.divar.ir/v8/posts-v2/web/wZraCiSl,2025-03-02 10:35:46.957
3,67c4363b98ac41ab4af1a11f,[https://s100.divarcdn.com/static/photo/neda/p...,فایل شخصی \nفرعی خوب \nشش طبقه سه واحدی ، طبقه...,90,فروش آپارتمان در یوسف‌آباد,NaN,۵۳متریکخواب اسدآبادی,"{'area': 53, 'production_year': 1380, 'rooms':...",2025-03-14T15:10:14.855149,,...,yousef-abad,,4,,7699999744,0,,wZrazsvQ,https://api.divar.ir/v8/posts-v2/web/wZrazsvQ,2025-03-02 10:43:03.563
4,67c43c81783c455ac2c92b1c,[https://s100.divarcdn.com/static/photo/neda/p...,✅️بهترین موقعیت و لوکیشن شمال فاطمی \n✅️دسترس...,90,فروش آپارتمان در یوسف‌آباد,"{'type': 'FUZZY', 'fuzzy_data': {'point': {'la...",فروش آپارتمان مسکونی ۹۸ متر ۲خواب شمال فاطمی,"{'area': 98, 'production_year': 1398, 'rooms':...",2025-04-02T14:17:01.056700,,...,yousef-abad,,1,,13200000000,0,,wZEbDtQH,https://api.divar.ir/v8/posts-v2/web/wZEbDtQH,2025-03-02 11:09:51.428


In [8]:
df.to_csv("../data/yousef-abad.csv", index=False)

# data preprocess

In [9]:
import ast

In [10]:
df = pd.read_csv("../data/yousef-abad.csv")
df.drop(columns="_id", inplace=True)

## auxilary functions

In [11]:
def column_summary(df, simple=True):
    summary_data = []

    for col_name in df.columns:
        col_summary = {}
        col_summary["col_name"] = col_name
        col_summary["col_dtype"] = df[col_name].dtype
        col_summary["num_of_nulls"] = df[col_name].isnull().sum()
        col_summary["num_of_non_nulls"] = df[col_name].notnull().sum()
        col_summary["null_present"] = col_summary["num_of_nulls"] > 0
        col_summary["num_of_distinct_values"] = df[col_name].nunique()
        col_summary["distinct_values_counts"] = df[col_name].value_counts().head(10).to_dict()
        if not simple:
            col_summary["min_value"] = df[col_name].min()
            col_summary["max_value"] = df[col_name].max()
            col_summary["mean"] = df[col_name].mean() if np.issubdtype(df[col_name].dtype, np.number) else None
            col_summary["median"] = df[col_name].median() if np.issubdtype(df[col_name].dtype, np.number) else None
        
        summary_data.append(col_summary)

    summary_df = pd.DataFrame(summary_data)
    return summary_df

In [12]:
def column_visualization(df):
    # Identify numerical columns
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    nonnumerical_columns = df.select_dtypes(exclude=[np.number]).columns
    
    # Perform univariate analysis on numerical columns
    for column in df.columns:
        if np.issubdtype(df[column].dtype, np.number) and len(df[column].unique()) > 10:
            plt.figure(figsize=(6, 3))
            sns.histplot(df[column], kde=True)
            plt.title(f'Histogram of {column}')
            plt.xlabel(column)
            plt.ylabel('Frequency')
            plt.show()

            plt.figure(figsize=(3, 4))
            sns.boxplot(df[column])
            plt.title(f'Boxplot of {column}')
            plt.xlabel(column)
            plt.ylabel('valeu')
            plt.show()

        elif df[column].nunique() < 10:
            plt.figure(figsize=(6, 3))
            ax = sns.countplot(df, x=column, hue=column, palette="tab10")
            plt.title(f'Count of {column}')
            plt.xlabel(column)
            plt.ylabel('Count')
            if not np.issubdtype(df[column].dtype, np.number):
                plt.xticks(rotation=45)

            # Annotate each bar with its count
            for p in ax.patches:
                ax.annotate(format(p.get_height(), '.0f'), 
                            (p.get_x() + p.get_width() / 2., p.get_height()), 
                            ha = 'center', va = 'center', 
                            xytext = (0, 5), 
                            textcoords = 'offset points')
            plt.show()
            print()

## location

In [13]:
def extract_lat_lon(location):
    if isinstance(location, str):
        try:
            location_dict = ast.literal_eval(location)
            if "exact_data" in location_dict:
                return (
                    "exact",
                    location_dict["exact_data"]["point"]["latitude"],
                    location_dict["exact_data"]["point"]["longitude"],
                )
            elif "fuzzy_data" in location_dict:
                return (
                    "fuzzy",
                    location_dict["fuzzy_data"]["point"]["latitude"],
                    location_dict["fuzzy_data"]["point"]["longitude"],
                )
        except (SyntaxError, ValueError):
            return np.nan, np.nan, np.nan
    return np.nan, np.nan, np.nan

In [14]:
df[["location_type", "latitude", "longitude"]] = df["location"].apply(lambda x: pd.Series(extract_lat_lon(x)))
df.drop(columns="location", inplace=True)

## features

In [15]:
import re

In [16]:
def process_features(column):
    if isinstance(column, str):
        try:
            return ast.literal_eval(column)
        except:
            return np.nan

    return np.nan

def convert_persian_digits(text):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    english_digits = "0123456789"
    translation_table = str.maketrans(persian_digits, english_digits)
    return text.translate(translation_table)

def parse_floor(value):
    if isinstance(value, int):
        return value, None
    
    value = convert_persian_digits(str(value))  
    match = re.match(r"(\d+) از (\d+)", value)
    
    if match:
        return int(match.group(1)), int(match.group(2))
    elif "زیرهمکف" in value:
        match = re.search(r"\d+", value)
        return -1, int(match.group()) if match else None
    elif "همکف" in value:
        match = re.search(r"\d+", value)
        return (0, int(match.group())) if match else (0, None)
    
    return None, None

In [17]:
features_df = df["features"].apply(lambda x: pd.Series(process_features(x)))
features_df.replace({"ندارد": False}, inplace=True)
features_df[["elevator", "parking", "warehouse", "balcony"]] = features_df[["elevator", "parking", "warehouse", "balcony"]].replace({"": True})
features_df["are_images_valid"] = features_df["are_images_valid"].replace({"بله": True}).fillna("False")
features_df["production_year"] = features_df["production_year"].replace({"قبل از ۱۳۷۰": "1369"}).astype(int)
features_df["rooms"] = features_df["rooms"].replace({"+۴": "5"}).astype(int)
features_df[["floor_number", "total_floors"]] = features_df["floor"].apply(lambda x: pd.Series(parse_floor(x)))
features_df.drop(columns="floor", inplace=True)

In [18]:
column_summary(features_df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,null_present,num_of_distinct_values,distinct_values_counts
0,area,float64,0,747,False,135,"{125.0: 44, 110.0: 43, 100.0: 33, 115.0: 27, 1..."
1,production_year,int64,0,747,False,35,"{1403: 200, 1402: 82, 1400: 41, 1401: 33, 1380..."
2,rooms,int64,0,747,False,5,"{2: 455, 3: 260, 1: 22, 4: 5, 5: 5}"
3,price,float64,0,747,False,292,"{20500000000.0: 17, 22000000000.0: 16, 2100000..."
4,unit_price,float64,0,747,False,529,"{200000000.0: 12, 150000000.0: 12, 190000000.0..."
5,unit_per_floor,float64,303,444,True,7,"{2.0: 286, 1.0: 117, 3.0: 28, 4.0: 7, 5.0: 3, ..."
6,deed_type,object,340,407,True,3,"{'تک‌برگ': 400, 'منگوله‌دار': 4, 'سایر': 3}"
7,building_direction,object,362,385,True,4,"{'جنوبی': 217, 'شمالی': 160, 'شرقی': 5, 'غربی'..."
8,rebuilt,object,2,745,True,2,"{'بازسازی نشده': 607, 'بازسازی شده': 138}"
9,elevator,object,1,746,True,2,"{True: 641, False: 105}"


In [19]:
df = pd.concat([df.drop(columns=["price", "features", "brand_model", "gender", "originality", "status", "credit", "rent"]), features_df], axis=1)

## price

In [20]:
def format_million_billion(x):
    """Converts large numbers to M (Million) or B (Billion)."""
    if x >= 1e9:
        return f"{x / 1e9:.1f}B"
    elif x >= 1e6:
        return f"{x / 1e6:.1f}M"
    else:
        return f"{x:.1f}"

In [21]:
df["price_h"] = df["price"].apply(format_million_billion)
df["unit_price_h"] = df["unit_price"].apply(format_million_billion)

In [22]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,null_present,num_of_distinct_values,distinct_values_counts
0,urls,object,50,697,True,697,{'['https://s100.divarcdn.com/static/photo/ned...
1,description,object,0,747,False,734,"{'زیر قیمت منطقه کمیسیون توافقی': 4, '⚜️ افتاب..."
2,district_id,int64,0,747,False,1,{90: 747}
3,district_name,object,0,747,False,1,{'فروش آپارتمان در یوسف‌آباد': 747}
4,title,object,0,747,False,710,{'فروش آپارتمان ۱۱۵ متری ۲ خوابه در یوسف‌آباد'...
5,unavailable_after,object,0,747,False,747,"{'2025-04-21T12:11:05.832014': 1, '2025-04-02T..."
6,business_ref,object,26,721,True,65,"{'MXYdIQan_ciZIrYqZ': 213, 'iOmnAFul_ZPkzwlXc'..."
7,business_type,object,0,747,False,2,"{'premium-panel': 721, 'personal': 26}"
8,cat_1,object,0,747,False,1,{'real-estate': 747}
9,cat_2,object,0,747,False,1,{'residential-sell': 747}
